# LUNA Basic Tutorial #1 - Calculating and filtering interactions

This tutorial covers: 
* Working with different Entry types (Mol, Ligand, etc)
* Using different filtering options to compute PPI, PLI, PNI, intra- or inter-molecular interactions
* Interaction granularity (proximal, covalent, non-covalent interactions)
* Applying filters on atomic groups or interactions using our built-in functions
* Creating a Pymol session to visualize interactions
* Plotting a Pymol session to depict hydrophobic contacts

### Set up system paths and import LUNA

In [1]:
LUNApath = '/srv/home/lshub/LUNA' #change this to point to LUNA directory

import sys
sys.path.append(LUNApath)

import luna

RDKit WARNING: [22:35:16] Enabling RDKit 2019.09.3 jupyter extensions
/srv/home/lshub/miniconda3/envs/luna-env/lib/python3.7/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')


**<ins>NOTE:</ins> in luna/util/default_values.py, change the OPENBABEL default location from 
    "/usr/bin/obabel" to location of obabel executable**

### Reading PDB file

#### Download from PDB

If the file does not exist locally, download from PDB by pdb_id

In [2]:
pdb_id = '3QQK'
input_path = "./inputs"
output_path = "./outputs"
luna.MyBio.util.download_pdb(pdb_id = pdb_id, output_path = input_path)

In [3]:
!ls ./inputs #confirm download

3QQK.pdb  X02.mol


#### Parse PDB file

In [4]:
pdb_file = '%s/%s.pdb' % (input_path, pdb_id)

# BioPython part: parse a pdb file into a structure object.
PDB_PARSER = luna.MyBio.PDB.PDBParser(PERMISSIVE = True, QUIET = True, FIX_ATOM_NAME_CONFLICT = False, 
                                      FIX_OBABEL_FLAGS = False)
structure = PDB_PARSER.get_structure(pdb_id, pdb_file)

# The function recover_entries_from_entity() allows you to recover all chains and ligands from a PDB file 
# and return a list of strings. Setting get_chains = False recovers ligands only.
for entry in luna.mol.entry.recover_entries_from_entity(structure, get_chains = False):
    print(entry)

3QQK:A:EDO:490
3QQK:A:EDO:491
3QQK:A:EDO:492
3QQK:A:EDO:493
3QQK:A:EDO:496
3QQK:A:X02:497


### Defining entries

Entries determine what interactions will be calculated. They can be ligands, chains, etc. 

In [5]:
from luna.mol.entry import Entry, MolEntry, ChainEntry

#### Manually

In [6]:
# ligand entry - X02
entry = luna.mol.entry.Entry("3QQK", "A", "X02", 497)
# or...
entry = luna.mol.entry.Entry.from_string("3QQK:A:X02:497", sep=":")

#### Read from mol file

In [7]:
entry = luna.mol.entry.MolEntry.from_mol_file("3QQK", "X02", "./inputs/X02.mol", 
                                              mol_obj_type='rdkit', is_multimol_file=False)

## Job setup

### Setting up parameters

In [8]:
opt = {}

opt['entries'] = [entry] #The list of entries to calculate 
opt['working_path'] = output_path # Where project results will be saved
opt['pdb_path'] = input_path # Path contiaining input PDB file - name must match that of entry pdb string.

opt['overwrite_path'] = False # Allows script to overwrite directoary - can remove files from previous project

opt['try_h_addition'] = True  # Define if you need to add Hydrogens or not
opt['ph'] = 7.4 # Controls the pH and how the hydrogens are going to be added - default 7.4
opt['amend_mol'] = False

opt['mol_obj_type'] = 'rdkit' # What type of mol object, options are 'rdkit' and 'openbabel' - default rdkit
opt['calc_ifp'] = False # Whether or not to calculate interaction fingerprint

opt['logging_enabled'] = True # Enable logger
opt['verbosity'] = 2 # How verbose is the logger

## Specifying what interactions to consider

### Interaction Configuration
This is the set of rules that define the distances and angles used to define the different types of interactions. The default configuration can be found in luna/mol/interaction/calc.py. This can be modified to support different interaction definitions.

In [9]:
from luna.mol.interaction.conf import DefaultInteractionConf

In [10]:
inter_conf = DefaultInteractionConf()
print(inter_conf.conf["max_da_dist_hb_inter"])

3.9


In [11]:
inter_conf.conf["max_da_dist_hb_inter"] = 4.0
print(inter_conf.conf["max_da_dist_hb_inter"])

4.0


In [12]:
opt['inter_conf'] = inter_conf

### Interaction Filters
InteractionFilters can be used to define what interactions to include in the result. Each filter has a specific set of default parameters. Modifying these filter parameters further specifies which of those interactions to include, ex. "ignore_self_inter" to ignore intra-molecular interactions.

In [13]:
from luna.mol.interaction.filter import InteractionFilter

In [14]:
pli_filter = InteractionFilter.new_pli_filter(ignore_self_inter = False) # protein-ligand interactions, including intra-molecular interactions
ppi_filter = InteractionFilter.new_ppi_filter() # protein-protein interactions
pni_filter = InteractionFilter.new_pni_filter() # protein-nucleotide interactions

### Interaction Calculator
Create an interaction calculator with the desired filter and add that to the project options. Like filters, calculaotrs have parameters to control interaction granularity. These include add_non_cov, add_cov, and add_proximal. 

In [15]:
from luna.mol.interaction.calc import InteractionCalculator

In [16]:
inter_calc = InteractionCalculator(inter_filter=pli_filter, add_proximal = False)
opt['inter_calc'] = inter_calc

## Run project
This produces a chunks directory, logs directory, and a results directory

In [17]:
proj_obj = luna.projects.LocalProject(**opt)
proj_obj.run()

[2021-04-04 22:35:19]    INFO            projects.py:305        Verbosity set to: 2 (WARNING).
[2021-04-04 22:35:19]    INFO            projects.py:305        Preparing project directory './outputs'.
[2021-04-04 22:35:19]    INFO                file.py:102        The directory './outputs' already exists, but it will not be cleared.
[2021-04-04 22:35:19]    INFO                file.py:102        The directory './outputs/results' already exists, but it will not be cleared.
[2021-04-04 22:35:19]    INFO            projects.py:305        Project directory './outputs' created successfully.


/srv/home/lshub/LUNA/luna/projects.py:267: UserWarning: Logging mode was set ON. Logging information will be saved from now on.
  warnings.warn("Logging mode was set ON. Logging information will be saved from now on.")


[2021-04-04 22:35:35]    PROGRESS                               100% [■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 1/1 [Avg: 15.28s/task; Errors: 0] - Entries processing.


In [18]:
!ls ./outputs

chunks	logs  project_v0.8.0.pkl.gz  results


### Evaluate results

The results for each individual entry can be accessed through the corresponding EntryResult

In [19]:
from luna.analysis.summary import count_interaction_types

In [20]:
entry_result = proj_obj.get_entry_results(entry)

In [21]:
print(f"Total number of interactions: {entry_result.interactions_mngr.size}")

Total number of interactions: 252


In [22]:
count_interaction_types(entry_result.interactions_mngr)

defaultdict(int,
            {'Atom overlap': 52,
             'Van der Waals': 53,
             'Van der Waals clash': 52,
             'Face-to-face pi-stacking': 2,
             'Aromatic bond': 22,
             'Hydrophobic': 29,
             'Single bond': 12,
             'Hydrogen bond': 9,
             'Weak hydrogen bond': 11,
             'Chalcogen-pi': 2,
             'Double bond': 4,
             'Displaced face-to-face pi-stacking': 2,
             'Edge-to-slope pi-stacking': 2})

#### Filter interactions by type

In [23]:
list(entry_result.interactions_mngr.filter_by_types(['Hydrogen bond']))

[<InteractionType: compounds=(<AtomGroup: [<ExtendedAtom: 3QQK/0/A/LEU-83/N>]>, <AtomGroup: [<ExtendedAtom: 3QQK/0/z/LIG-9999/N2>]>) type=Hydrogen bond>,
 <InteractionType: compounds=(<AtomGroup: [<ExtendedAtom: 3QQK/0/A/HOH-321/O>]>, <AtomGroup: [<ExtendedAtom: 3QQK/0/A/X02-497/O18>]>) type=Hydrogen bond>,
 <InteractionType: compounds=(<AtomGroup: [<ExtendedAtom: 3QQK/0/A/HOH-321/O>]>, <AtomGroup: [<ExtendedAtom: 3QQK/0/z/LIG-9999/O1>]>) type=Hydrogen bond>,
 <InteractionType: compounds=(<AtomGroup: [<ExtendedAtom: 3QQK/0/z/LIG-9999/N3>]>, <AtomGroup: [<ExtendedAtom: 3QQK/0/A/GLU-81/O>]>) type=Hydrogen bond>,
 <InteractionType: compounds=(<AtomGroup: [<ExtendedAtom: 3QQK/0/A/X02-497/N4>]>, <AtomGroup: [<ExtendedAtom: 3QQK/0/A/LEU-83/O>]>) type=Hydrogen bond>,
 <InteractionType: compounds=(<AtomGroup: [<ExtendedAtom: 3QQK/0/A/LEU-83/N>]>, <AtomGroup: [<ExtendedAtom: 3QQK/0/A/X02-497/N6>]>) type=Hydrogen bond>,
 <InteractionType: compounds=(<AtomGroup: [<ExtendedAtom: 3QQK/0/A/ASP-145/N

### Visualizing interactions in pymol

In [24]:
from luna.mol.interaction.view import InteractionViewer

In [25]:
entry.pdb_file = './inputs/3QQK.pdb' # set entry PDB file

all_tuples = [(entry, entry_result.interactions_mngr)]
pse_file = "./outputs/3QQK-visualization-all.pse"

inter_view = InteractionViewer()
inter_view.new_session(all_tuples, pse_file)

 PyMOL not running, entering library mode (experimental)
 Applying pse_export_version=1.800 compatibility


### Visualizing specific types of interactions

In [26]:
hphobe_inter = list(entry_result.interactions_mngr.filter_by_types(['Hydrophobic']))

hp_tuples = [(entry, hphobe_inter)]
pse_file = "./outputs/3QQK-visualization-hydrophobic.pse"

inter_view = InteractionViewer()
inter_view.new_session(hp_tuples, pse_file)

 Applying pse_export_version=1.800 compatibility


## Save full project or individual entry result

In [27]:
proj_obj.save(output_file = "./outputs/3QQK_X02_proj")
entry_result.save(output_file = "./outputs/3QQK_X02_entry")